In [ ]:
import sys
sys.path.append("..")
from f16lib.systems import F16Simple, F16AcasShield, F16Shield, F16AirspeedSimple


import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
from IPython.display import Image

import pathlib

plot_fname = f"pub-sub-plot.png"

# plot configuration pub/sub diagram as a file -- proj specicies a dot executbale and -Gdpi is a valid dot
# argument to change the image resolution
my_conf = F16AcasShield()
my_conf.plot_config(fname=pathlib.Path(plot_fname).resolve(), prog=["dot", "-Gdpi=400"])

# display written file to notebook
Image(plot_fname, height=600)

In [ ]:
def air_collision_condition(ctraces):
        """ground collision premature termination condition
        """
        # get the aircraft states
        sa, sb = ctraces['plant']['states'], ctraces['plant_intruder']['states']
        if sa and sb:
            # look at distance between last state
            return (np.linalg.norm(np.array(sa[-1][9:11]) - np.array(sb[-1][9:11]))) < 100

In [ ]:
# set the scenario states
my_system = F16AcasShield()
f16_xequil = [450.0,
              0.03887505597600522,
              0.0,
              0.0,
              0.03887505597600522,
              0.0,
              0.0,
              0.0,
              0.0,
              0.0,
              0.0,
              1000.0,
              9.05666543872074]
my_system.set_state('plant', [800.0, 0.0389, 0.0, 0.0, 0.0389, 0.0, 0.0, 0.0, \
                        0.0, 0.0, 0.0, 1000.0, 9.0567])
my_system.set_state('intruder_plant', [800.0, 0.0389, 0.0, 0.0, 0.0389, np.pi, 0.0, 0.0, \
                        0.0, 10000.0, 500.0, 1000.0, 9.0567])
my_system.set_component_iv('autopilot', 'inputs_own', [800.0, 0.0389, 0.0, 0.0, 0.0389, 0.0, 0.0, 0.0, \
                        0.0, 0.0, 0.0, 1000.0, 9.0567])
my_system.set_component_iv('autopilot', 'inputs_intruder0', [800.0, 0.0389, 0.0, 0.0, 0.0389, np.pi, 0.0, 0.0, \
                        0.0, 10000.0, 500.0, 1000.0, 9.0567])

In [ ]:
trajs = my_system.simulate_tspan((0, 35.0), show_status=False)

In [ ]:
plt.plot(*np.array(trajs["plant"].states)[:, 9:11].T)

In [ ]:
from joblib import Parallel, delayed

In [ ]:
from time import perf_counter
from contextlib import contextmanager
import tqdm


def benchmark_task(i):
    my_system = F16AcasShield()
    my_system.set_state('plant', [800.0, 0.0389, 0.0, 0.0, 0.0389, 0.0, 0.0, 0.0, \
                            0.0, 0.0, 0.0, 1000.0, 9.0567])
    my_system.set_state('intruder_plant', [800.0, 0.0389, 0.0, 0.0, 0.0389, np.pi, 0.0, 0.0, \
                            0.0, 10000.0, -500.0+i*100, 1000.0, 9.0567])
    trajs = my_system.simulate_tspan((0, 35.0), show_status=False)
    return trajs


@contextmanager
def catchtime() -> float:
    start = perf_counter()
    yield lambda: perf_counter() - start

In [ ]:
with catchtime() as t:
    trajs = [benchmark_task(i) for i in tqdm.tqdm(range(10))]
print(f"[for] Execution Time: {t():.4f} secs")

with catchtime() as t:
    trajsp = Parallel(n_jobs=10)(delayed(benchmark_task)(i) for i in range(10))
print(f"[parallel for] Execution Time: {t():.4f} secs")

In [ ]:
for t in trajsp[8:]:
    plt.plot(*np.array(t["plant"].states)[:, 9:11].T)